# API Basics
DOC-Server API is a RESTfull Http API.  
## Environments
There are 2 environments are available:  
* **Quality (QA)**  
Use this environment for testing purposes. Services [APi OAS3 here](https://doc-server-qa.saphety.com/Doc.WebApi.Services/api/index.html)  
* **Production**  
Production environment. Services [APi OAS3 here](https://doc-server-qa.saphety.com/Doc.WebApi.Services/api/index.html)

## Authentication
All services require JWT autorization token. This token is used as a bearer token on each request header.  
Use your credentials (username, password) to get a token at:   
```  
https://<ServerBaseAddress>/api/Account/token  
```   

In [24]:
#Set Environment
#Integration
server_base_adress = "doc-server-int.saphety.com/Doc.WebApi.Services"
#Quality
#server_base_adress = "doc-server-qa.saphety.com/Doc.WebApi.Services"
#Production
#server_base_adress = "doc-server.saphety.com/Doc.WebApi.Services"

In [25]:
import requests
import json

# DOC-Server account service url
service_url = "https://" + server_base_adress + "/api/Account/token"

# Example of username and password
username = 'user'
password = 'pass'

# Auhtentication data goes in payload as json
payload = {
      'Username': username,
      'Password': password
}
# Payload goes in json, serialize the payloal object to json
request_data=json.dumps(payload)
# Indicate in header that payload is json
headers = {
    'content-type': 'application/json'
    }
# POST request to get a token
response = requests.request("POST", service_url, data=request_data, headers=headers)

# Formating the response to json for visualization purposes only
json_response = json.loads(response.text)
print(json.dumps(json_response, indent=4))

{
    "CorrelationId": "62509d9a-8cc2-4823-a77b-3d586141d454",
    "IsValid": false,
    "Errors": [
        {
            "Code": "INVALID_AUTHENTICATION_DATA",
            "Field": "Username",
            "Values": [
                "user"
            ]
        }
    ],
    "Data": null
}


In [19]:
# Your token is at:
token = json_response["Data"];
print (token)

None


### bearer token
The token obtained in the ```/api/Account/token``` service is to be used has a bearer token in the http header  
```Javascript
POST https://<Service URL> HTTP/1.1
content-type: application/json
authorization: bearer <token goes here>
```

## Services response structure
All services return a similar data structure (**response message**).  
When a request is well formed and the authentication data is correct the system responds with a message envelope as follows: 
```Javascript
{
	"CorrelationId": "<GUID>", /* for correlation purposes (ex: contact support for troubleshooting) */
	"IsValid": true,           /* false in case of erros */
	"Errors": [],              /* if empty is a good signal */
	"Data": "<Service Response Data>"   /* the data retuned ex: token, invoice status .. dependent on the service called */
}
```

### The Errors array
Services always return ```http 200``` even if the there are errors on the result.   
A service only performed its action succeffully if the ```Isvalid``` property in result is ```true```  
When the  the ```Isvalid``` property in result is ```false``` then a list of errors shouls be present in the ```Errors``` array.  
### The Error object
The ```Errors``` array is a list o ```Error``` objects:  
```Javascript   
{
    "Code": "INVALID_ENUM", /* The error code */
    "Field": "DocumentType", /* When present, indicates the property responsible for the error */
    "Values": [ "INVOIC" ] /* List of values that help to better error context; in this case INVOIC is not a valid  DocumentType */
}
```

## Pagination services
Some query services can return a large set of data (ex: _searching documents_)  
These services are tipically designated as _searching services_.  
There 3 properties in common with these services:
* They provide a mechanism of filtering data via a proprety ```RestrictionCriteria```  
* They provide a mechanism _pagination_ retrieving data  
* They provide a counting mecanism as a different service that receives the same ```RestrictionCriteria```  
### Pagination data

Pagination is the the information passed to the service in order to obtain a sub-set (page) of the data.  
The pagination info is as following:

```Javascript
{
  "RestrictionCriteria": {}, /* Restriction criteria depends on each search service */
  "OrderBy": null, /* Optional: indicates the property that server should order the data. By default is the Cration Date (when applicable) */
  "OrderDescending": true, /* Oprional: related to OrderBy property */
  "PageNumber": 0,  /* Starts at 0. Indicates the sub-set number (page) to be retrieved; Must be used in accordance with RowsPerPage */
  "RowsPerPage": 20 /* Number of records to be set on each page, Client only receives one page at a time (Page Number) */
}

```
### Restriction criteria
The restiction criteria depends on the service that is beeing called. When searching documents we can filter by document number but when searching companies a filter by vat number is more suitable.  
The following is a sample of a restriction criteria for searching users  

```Javascript
{
  "RestrictionCriteria": {
    "Username": null,
    "Name": null,
    "Email": "some@email.com"
  }, 
  "OrderBy": null, 
  "OrderDescending": true, 
  "PageNumber": 0, 
  "RowsPerPage": 20 
}
```
### Count service
All _searching services_ have a correspondent ```count``` service.   
The ```count``` service returns the number of total records that match the ```RestrictionCriteria```